**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 4.0MB/s 
     |████████████████████████████████| 51kB 4.3MB/s 


Read the RDF file

In [2]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=N80aafa8ca1f7471087c37b7140ce1db0 (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [11]:
ns = Namespace("http://somewhere#")
vcard = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")

#RDFLib
for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
    print(s)
for a,b,c in g.triples((None, RDFS.subClassOf, s)): 
  print(a)

#SPARQL
from rdflib.plugins.sparql import prepareQuery
q1 = prepareQuery('''
  SELECT 
    ?Subject
  WHERE { 
    ?Subject rdfs:subClassOf/rdfs:subClassOf* ns:Person.
  }
  ''',
  initNs = {"rdfs":RDFS, "ns":ns}
)

for r in g.query(q1):
  print(r)

http://somewhere#Researcher
http://somewhere#PhDstudent
(rdflib.term.URIRef('http://somewhere#Researcher'),)
(rdflib.term.URIRef('http://somewhere#PhDstudent'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [12]:

for s,p,o in g.triples((None, RDF.type, ns.Person)):
  print(s)

for subClasses,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s,p,o in g.triples((None, RDF.type, subClasses)):
    print(s)

#SPARQL

q2 = prepareQuery('''
  SELECT 
    ?Subject 
  WHERE {
      ?Subject rdf:type ns:Person.
  }
  ''',
  initNs = {"ns":ns}
)
for r in g.query(q2):
  print(r)


http://somewhere#JohnSmith
http://somewhere#SaraJones
http://somewhere#JaneSmith
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#SaraJones'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [19]:

for s,p,o in g.triples((None, RDF.type, ns.Person)):
  for s1,p1,o1 in g.triples((s,None,None)):
    print(s1,p1,o1)

for subClasses,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s,p,o in g.triples((None, RDF.type, subClasses)):
    for s1,p1,o1 in g.triples((s,None,None)):
      print(s1,p1,o1)

#SPARQL 

q3 = prepareQuery('''
  SELECT 
    ?Subject ?p ?o
  WHERE {
      ?subclasses rdfs:subClassOf/rdfs:subClassOf* ns:Person.
      ?Subject rdf:type ?subclasses.
      ?Subject ?p ?o
  }
  ''',
  initNs = {"ns":ns}
)

for r in g.query(q3):
  print(r)

http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Researcher
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN jane Smith
http://somewhere#JaneSmith http://somewher